In [45]:
# To Do: Create global config object for things that are not tied to a
#        particular dataset configuration but may still be changeable, e.g.
#        user messages related to input validation. Or maybe not this is an
#        interactive notebook not some compiled software.

import ipywidgets as widgets
import json

from IPython.display import display


with open('./default/default_employees.json') as default_configuration_json:
    config = json.load(default_configuration_json)


l3_config_dataset_details = config['overview']['dataset_details']

l3_config_organisation_size   = config['employee_properties']['organisation_size']
l3_config_employment_type_mix = config['employee_properties']['employment_type_mix']

In [46]:
# Level 3: Standard Properties

l3_box_layout = widgets.Layout(
      display         = 'flex'
    , flex_flow       = 'column'
    , align_content   = 'center'
    , justify_content = 'center'
)

l3_text_layout = widgets.Layout(
    display           = 'flex'
  , justify_content   = 'center'
  , width             = '25%'
)

l3_input_layout = widgets.Layout(
    width             = 'max_content'
)

l3_input_style = {
    'description_width': '50%'
}

In [47]:
# Level 3: Overview / Dataset Details
l3_dataset_details_configuration_name = widgets.Text(
    value       = l3_config_dataset_details['configuration_name']
  , description = 'Configuration Name'
  , disabled    = False
  , layout      = l3_input_layout
  , style       = l3_input_style
)

l3_dataset_details_dataset_configuration_last_modified = widgets.Text(
    value       = l3_config_dataset_details['last_modified']
  , description = 'Last Modified'
  , disabled    = True
  , layout      = l3_input_layout
  , style       = l3_input_style
)

l3_dataset_details_dataset_name = widgets.Text(
    value       = l3_config_dataset_details['dataset_name']
  , description = 'Dataset Name'
  , disabled    = True
  , layout      = l3_input_layout
  , style       = l3_input_style
)

l3_dataset_details = widgets.Box(
    children = [l3_dataset_details_configuration_name] +
               [l3_dataset_details_dataset_configuration_last_modified] +
               [l3_dataset_details_dataset_name]
  , layout = l3_box_layout
)

In [48]:
# Level 2: Overview
# Maybe down the track we will want additional information in level 2 overview.
l2_overview = l3_dataset_details

In [49]:
# Level 3: General / Organisation Size
l3_general_organisation_size = widgets.BoundedIntText(
      value       = l3_config_organisation_size['value']
    , min         = 100
    , max         = 100000
    , description = l3_config_organisation_size['description']
    , disabled    = False
    , layout      = l3_input_layout
    , style       = l3_input_style
)

In [50]:
# Level 3: Employee Properties / Employment Type Mix
# To Do:  Create functionality for users to add/remove/modify employment types descriptions.
def l3_employment_type_mix_on_value_change(change):
  if change['old'] != change['new']:
    l3_employment_type_mix_total.value += change['new'] - change['old']

  if l3_employment_type_mix_total.value == 100:
    l3_employment_type_mix_user_message.value = 'Employment types total 100% Good to go!'
  elif l3_employment_type_mix_total.value > 100:
    l3_employment_type_mix_user_message.value = 'Employment types total over 100% Please correct mix.'
  else:
    l3_employment_type_mix_user_message.value = 'Employment types total under 100% Please correct mix.'

def l3_employment_type_mix_values_to_json():
  dumps({input.description: input.value / 100 for input in l3_employment_type_mix_body.children})


l3_employment_type_mix        = widgets.Box(layout = l3_box_layout)
l3_employment_type_mix_header = widgets.Box(layout = l3_box_layout)
l3_employment_type_mix_body   = widgets.Box(layout = l3_box_layout)


l3_employment_type_mix_user_message = widgets.Label(value = '')

l3_employment_type_mix_total = widgets.IntText(
    description = 'Total'
  , value       = 0
  , disabled    = True
  , layout      = l3_input_layout
  , style       = l3_input_style
)

l3_employment_type_mix_header.children = [l3_employment_type_mix_user_message] + [l3_employment_type_mix_total]

l3_employment_type_mix_body.children = [widgets.BoundedIntText(
      description = key
    , value       = value * 100
    , min         = 0
    , max         = 100
    , step        = 1
    , disabled    = False
    , layout      = l3_input_layout
    , style       = l3_input_style
) for i, (key, value) in enumerate(l3_config_employment_type_mix['values'].items())]


for input in l3_employment_type_mix_body.children:
  input.observe(l3_employment_type_mix_on_value_change, names='value')


l3_employment_type_mix_total.value = sum([input.value for input in l3_employment_type_mix_body.children])
l3_employment_type_mix_on_value_change({'old': 0, 'new': 0})


l3_employment_type_mix.children = [l3_employment_type_mix_header] + [l3_employment_type_mix_body]

In [51]:
# Level 2: Employee Properties
l2_employee_properties = widgets.Accordion(
      children = [l3_general_organisation_size, l3_employment_type_mix]
    , titles   = ['Organisation Size', 'Employment Type Mix (percentage)']
)

In [52]:
l1_tab = widgets.Tab(
    children = [
          l2_overview
        , l2_employee_properties
        , l2_employee_properties
        , l2_employee_properties
    ]
  
  , titles   = [
      'Overview'
    , 'Employee Properties'
    , 'Divisions'
    , 'Teams'
  ]
)

l1_tab